In [6]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.preprocessing import scale

In [5]:
# Import and preprocess data

raw_data = pd.read_csv('data/Advertising.csv').drop('Unnamed: 0', axis=1)
raw_data.head()

scaled_data = scale(raw_data)


NameError: name 'preprocessing' is not defined